### Jupyter CodeStats 

Code:Stats is een online service die je aan je IDE's kunt koppelen, zodat je kunt bijhouden in welke programmeertalen je het meeste werkt. Ik heb ontdekt dat dit ook aan Jupyter gekoppeld kan worden, dus vandaar dit experiment. Kan ik in dit notebook mijn werk terugzien via de Code::Stats API?

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
result = pd.read_json("https://codestats.net/api/users/reinonlein")

In [3]:
xp_per_day = (result['dates']
              .dropna()
              .reset_index()
              .sort_values('index')
              .assign(datum = lambda d: pd.to_datetime(d['index']))
              .drop('index', axis = 1)
              .rename(columns={'dates': 'xp'})
            )

In [4]:
xp_per_day

xp      datum
60     29.0 2020-05-16
84   3091.0 2020-05-17
37   2170.0 2020-05-18
25   1417.0 2020-05-19
4     731.0 2020-05-20
..      ...        ...
63   7856.0 2020-10-18
58  11081.0 2020-10-19
51   2194.0 2020-10-20
57   2085.0 2020-10-21
10   3463.0 2020-10-22

[100 rows x 2 columns]

In [53]:
daterange_df = pd.DataFrame({'datum': pd.date_range(min(xp_per_day['datum']), pd.to_datetime('today'))})

In [56]:
daily_xp = daterange_df.merge(xp_per_day, on='datum', how='left')

In [57]:
fig = px.bar(daily_xp, x="datum", y="xp", title='CodeStats XP per day')
fig.show()

In [58]:
fig = px.line(daily_xp, x="datum", y="xp", title='CodeStats XP per day')
fig.show()

In [62]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_xp['datum'],
    y=daily_xp['xp'],
    connectgaps=False # override default to connect the gaps
))


fig.show()

LEVEL_FACTOR = 0.025

// Get level for given XP  
int get_level(float xp) {  
  return to_int(floor(LEVEL_FACTOR * sqrt(xp)))  
}

In [16]:
df = (pd.read_json("https://codestats.net/api/users/reinonlein")
      .reset_index()
      [['index', 'languages']]
      .dropna()
      .assign(xp = lambda d: d['languages'].apply(pd.Series)['xps'])
      .assign(new_xp = lambda d: d['languages'].apply(pd.Series)['new_xps'])
      .reset_index()
      .drop(['languages', 'level_0'], axis=1)
      .rename(columns={'index': 'language'})
      .sort_values('xp', ascending=False)
      .assign(level = lambda d: np.floor(0.025 * np.sqrt(d['xp'])).astype(int))

)

df

language      xp  new_xp  level
1         Dart  114884       0      8
3         HTML   59150     394      6
9     Markdown   13702       0      2
12      Python    8112    3098      2
6         Java    7225       0      2
13         SQL    4050       0      1
10  Plain text    3876       0      1
7   JavaScript    2776       0      1
15        YAML    2727       0      1
16    scminput    2712       0      1
14         XML     612       0      0
2       Groovy     478       0      0
5         JSON     431       0      0
4       Ignore     383       0      0
11  Properties     257       0      0
8       Kotlin     233       0      0
0          CSS     199       0      0

In [12]:
fig = px.bar(df.sort_values('xp'), x="xp", y="language", orientation='h', title = 'Current XP per language')
fig.show()

In [19]:
fig = px.pie(df, values='level', names='language')
fig.show()